In [2]:
from datetime import datetime, timedelta
import pandas as pd
import requests, json
from pandas import json_normalize

In [3]:
today = datetime.now()

In [4]:
today.strftime('%Y%m%y')

tyear = today.year
tmon = today.month
tday = today.day

# tmon = 4
c_date = f'{tyear:02d}-{tmon:02d}-{tday:02d}'


ser_day = datetime.strptime(c_date, '%Y-%m-%d')


In [5]:
import calendar

# 0이면 월요일, 6이면 일요일이 시작일이다.
start_day_week, end_day = calendar.monthrange(tyear, tmon)
print(start_day_week, end_day)

# 해당 달의 첫날의 datetime형식으로 변환한다.
start_date = datetime.strptime(f'{tyear:02d}-{tmon:02d}-01', '%Y-%m-%d')
end_date = datetime.strptime(f'{tyear:02d}-{tmon:02d}-{end_day:02d}', '%Y-%m-%d')
print(start_date, end_date)


0 31
2023-05-01 00:00:00 2023-05-31 00:00:00


In [6]:
start_date.strftime('%Y-%m-%d')

'2023-05-01'

In [7]:
pd.date_range(start_date.strftime('%Y-%m-%d'), periods=3)

DatetimeIndex(['2023-05-01', '2023-05-02', '2023-05-03'], dtype='datetime64[ns]', freq='D')

In [8]:
# 전 달 구하기
lines, style = [], []
l_dict = dict()

before_month_day = 0
if start_day_week < 6:
    before_month_day = (start_date - timedelta(start_day_week+1))
    print(before_month_day)

    lines.extend(pd.date_range(before_month_day.strftime('%Y-%m-%d'), periods=start_day_week+1))
    style = ['0.5' for i in range(start_day_week+1)]
    # for i in range(start_day_week+1):
    #     style.append('0.5')
        

print(lines)
print(style)
    


2023-04-30 00:00:00
[Timestamp('2023-04-30 00:00:00')]
['0.5']


In [9]:
# 현재 달
lines.extend(pd.date_range(start_date.strftime('%Y-%m-%d'), periods=end_day))
style.extend(['' for i in range(end_day)])

In [10]:
# lines.extend(range(1, end_day+1))
print(lines)
print(len(style))

[Timestamp('2023-04-30 00:00:00'), Timestamp('2023-05-01 00:00:00'), Timestamp('2023-05-02 00:00:00'), Timestamp('2023-05-03 00:00:00'), Timestamp('2023-05-04 00:00:00'), Timestamp('2023-05-05 00:00:00'), Timestamp('2023-05-06 00:00:00'), Timestamp('2023-05-07 00:00:00'), Timestamp('2023-05-08 00:00:00'), Timestamp('2023-05-09 00:00:00'), Timestamp('2023-05-10 00:00:00'), Timestamp('2023-05-11 00:00:00'), Timestamp('2023-05-12 00:00:00'), Timestamp('2023-05-13 00:00:00'), Timestamp('2023-05-14 00:00:00'), Timestamp('2023-05-15 00:00:00'), Timestamp('2023-05-16 00:00:00'), Timestamp('2023-05-17 00:00:00'), Timestamp('2023-05-18 00:00:00'), Timestamp('2023-05-19 00:00:00'), Timestamp('2023-05-20 00:00:00'), Timestamp('2023-05-21 00:00:00'), Timestamp('2023-05-22 00:00:00'), Timestamp('2023-05-23 00:00:00'), Timestamp('2023-05-24 00:00:00'), Timestamp('2023-05-25 00:00:00'), Timestamp('2023-05-26 00:00:00'), Timestamp('2023-05-27 00:00:00'), Timestamp('2023-05-28 00:00:00'), Timestamp('20

In [11]:
# 빈 칸에 다음 달 날짜 구하기
nday = calendar.weekday(tyear, tmon, end_day)
next_cnt = 0
if nday == 6: next_cnt = 6
elif nday < 5: next_cnt = 5 - nday

# 마지막 날짜 구하기
lines.extend(pd.date_range((end_date+timedelta(1)).strftime('%Y-%m-%d'), periods=next_cnt))
style.extend(['0.5' for i in range(next_cnt)])

# lines.extend(range(1, next_cnt+1))
# print(lines)

In [12]:
l_dt = pd.DataFrame({'locdate':lines, 'opacity': style})
l_dt['day'] = l_dt.locdate.apply(lambda x: int(x.strftime('%d')))
l_dt.head(7)

,locdate,opacity,day
0,2023-04-30,0.5,30
1,2023-05-01,,1
2,2023-05-02,,2
3,2023-05-03,,3
4,2023-05-04,,4
5,2023-05-05,,5
6,2023-05-06,,6


In [13]:
for n in lines[:5]:
    print(n)

2023-04-30 00:00:00
2023-05-01 00:00:00
2023-05-02 00:00:00
2023-05-03 00:00:00
2023-05-04 00:00:00


In [14]:
with open('../static/key/holidays.txt') as f:
    apikey = f.read()

- 공휴일

In [15]:
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?_type=json&numOfRows=50&solYear='
base_url += str(tyear) + '&ServiceKey=' + apikey

In [16]:
result = requests.get(base_url)
if result.status_code == 200:
    res = result.text

print(res)

{"response":{"header":{"resultCode":"00","resultMsg":"NORMAL SERVICE."},"body":{"items":{"item":[{"dateKind":"01","dateName":"1월1일","isHoliday":"Y","locdate":20230101,"seq":1},{"dateKind":"01","dateName":"설날","isHoliday":"Y","locdate":20230121,"seq":1},{"dateKind":"01","dateName":"설날","isHoliday":"Y","locdate":20230122,"seq":1},{"dateKind":"01","dateName":"설날","isHoliday":"Y","locdate":20230123,"seq":1},{"dateKind":"01","dateName":"대체공휴일","isHoliday":"Y","locdate":20230124,"seq":1},{"dateKind":"01","dateName":"삼일절","isHoliday":"Y","locdate":20230301,"seq":1},{"dateKind":"01","dateName":"어린이날","isHoliday":"Y","locdate":20230505,"seq":1},{"dateKind":"01","dateName":"부처님오신날","isHoliday":"Y","locdate":20230527,"seq":1},{"dateKind":"01","dateName":"대체공휴일","isHoliday":"Y","locdate":20230529,"seq":1},{"dateKind":"01","dateName":"현충일","isHoliday":"Y","locdate":20230606,"seq":1},{"dateKind":"01","dateName":"광복절","isHoliday":"Y","locdate":20230815,"seq":1},{"dateKind":"01","dateName":"추석","isHol

In [17]:
res = json.loads(res)
res.keys()

dict_keys(['response'])

In [18]:
res['response']['body']['items']['item']

[{'dateKind': '01',
  'dateName': '1월1일',
  'isHoliday': 'Y',
  'locdate': 20230101,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '설날',
  'isHoliday': 'Y',
  'locdate': 20230121,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '설날',
  'isHoliday': 'Y',
  'locdate': 20230122,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '설날',
  'isHoliday': 'Y',
  'locdate': 20230123,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '대체공휴일',
  'isHoliday': 'Y',
  'locdate': 20230124,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '삼일절',
  'isHoliday': 'Y',
  'locdate': 20230301,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '어린이날',
  'isHoliday': 'Y',
  'locdate': 20230505,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '부처님오신날',
  'isHoliday': 'Y',
  'locdate': 20230527,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '대체공휴일',
  'isHoliday': 'Y',
  'locdate': 20230529,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '현충일',
  'isHoliday': 'Y',
  'locdate': 20230606,
  'seq': 1},
 {'dateKind': '01',
  'd

In [19]:
holidays = json_normalize(res['response']['body']['items']['item'])
holidays

,dateKind,dateName,isHoliday,locdate,seq
0,01,1월1일,Y,20230101,1
1,01,설날,Y,20230121,1
2,01,설날,Y,20230122,1
3,01,설날,Y,20230123,1
4,01,대체공휴일,Y,20230124,1
5,01,삼일절,Y,20230301,1
6,01,어린이날,Y,20230505,1
7,01,부처님오신날,Y,20230527,1
8,01,대체공휴일,Y,20230529,1
9,01,현충일,Y,20230606,1


- 24절기

In [20]:
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/get24DivisionsInfo?_type=json&numOfRows=50&solYear='
base_url += str(tyear) + '&ServiceKey=' + apikey
result = requests.get(base_url)
if result.status_code == 200:
    res = result.text

res = json.loads(res)
res.keys()

dict_keys(['response'])

In [21]:
json_normalize(res['response']['body']['items']['item'])[['dateKind', 'dateName', 'isHoliday', 'locdate']]

,dateKind,dateName,isHoliday,locdate
0,03,소한,N,20230106
1,03,대한,N,20230120
2,03,입춘,N,20230204
3,03,우수,N,20230219
4,03,경칩,N,20230306
5,03,춘분,N,20230321
6,03,청명,N,20230405
7,03,곡우,N,20230420
8,03,입하,N,20230506
9,03,소만,N,20230521


- 기념일

In [22]:
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getAnniversaryInfo?_type=json&numOfRows=80&solYear='
base_url += str(tyear) + '&ServiceKey=' + apikey
result = requests.get(base_url)
if result.status_code == 200:
    res = result.text

res = json.loads(res)
dt = pd.concat([holidays, json_normalize(res['response']['body']['items']['item'])])


In [23]:
res['response']['body']['items']['item']

[{'dateKind': '02',
  'dateName': '2·28 민주운동 기념일',
  'isHoliday': 'N',
  'locdate': 20230228,
  'seq': 1},
 {'dateKind': '02',
  'dateName': '납세자의 날',
  'isHoliday': 'N',
  'locdate': 20230303,
  'seq': 1},
 {'dateKind': '02',
  'dateName': '3·8민주의거 기념일',
  'isHoliday': 'N',
  'locdate': 20230308,
  'seq': 1},
 {'dateKind': '02',
  'dateName': '3·15 의거 기념일',
  'isHoliday': 'N',
  'locdate': 20230315,
  'seq': 1},
 {'dateKind': '02',
  'dateName': '상공의 날',
  'isHoliday': 'N',
  'locdate': 20230315,
  'seq': 2},
 {'dateKind': '02',
  'dateName': '서해수호의 날',
  'isHoliday': 'N',
  'locdate': 20230324,
  'seq': 1},
 {'dateKind': '02',
  'dateName': '4·3희생자 추념일',
  'isHoliday': 'N',
  'locdate': 20230403,
  'seq': 1},
 {'dateKind': '02',
  'dateName': '식목일',
  'isHoliday': 'N',
  'locdate': 20230405,
  'seq': 1},
 {'dateKind': '02',
  'dateName': '예비군의 날',
  'isHoliday': 'N',
  'locdate': 20230407,
  'seq': 1},
 {'dateKind': '02',
  'dateName': '보건의 날',
  'isHoliday': 'N',
  'locdate': 202304

In [24]:
dt[30:45]

,dateKind,dateName,isHoliday,locdate,seq
13,02,장애인의 날,N,20230420,1
14,02,과학의 날,N,20230421,1
15,02,정보통신의 날,N,20230422,1
16,02,법의 날,N,20230425,1
17,02,충무공 이순신 탄신일,N,20230428,1
18,02,근로자의 날,N,20230501,1
19,02,어버이 날,N,20230508,1
20,02,유권자의 날,N,20230510,1
21,02,동학농민혁명 기념일,N,20230511,1
22,02,식품안전의 날,N,20230514,1


- 잡절


In [25]:
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getSundryDayInfo?_type=json&numOfRows=50&solYear='
base_url += str(tyear) + '&ServiceKey=' + apikey
result = requests.get(base_url)
if result.status_code == 200:
    res = result.text

res = json.loads(res)
json_normalize(res['response']['body']['items']['item'])

,dateKind,dateName,isHoliday,locdate,seq
0,04,정월대보름,N,20230205,1
1,04,한식,N,20230406,1
2,04,단오,N,20230622,1
3,04,초복,N,20230711,2
4,04,중복,N,20230721,1
5,04,말복,N,20230810,1
6,04,칠석,N,20230822,1


- 한 번에 구하기

In [26]:
def is_info_day(in_list, s_date, s_datename):
    # print(type(lists))
    for i in range(len(in_list)):
        # print( in_list[i]['locdate'], s_date)
        if in_list[i]['locdate'] == s_date:
            in_list[i]['dateName'] = in_list[i]['dateName'] + ' ' + s_datename
            # print(in_list[i]['dateName'])
            return True
    return False

In [27]:
tyear = 2023
with open('../static/key/holidays.txt') as f:
    apikey = f.read()
    
f_list = ['getRestDeInfo', 'getAnniversaryInfo', 'get24DivisionsInfo', 'getSundryDayInfo']
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/' #getSundryDayInfo?_type=json&numOfRows=50&solYear='
param1 = '?_type=json&numOfRows=80&solYear='

holidays = None
in_list = []

for name in f_list:
    url = base_url + name + param1 + str(tyear) + '&ServiceKey=' + apikey
    ress = ''

    try:
        result = requests.get(url) 
        if result.status_code == 200:
            ress = result.text
    except:
        print('CODE = ', result.status_code)
        pass

    if ress: 
        ress = json.loads(ress)['response']['body']['items']['item']
        for i in range(len(ress)):
            # print(ress[i]['locdate'])
            if is_info_day(in_list, ress[i]['locdate'], ress[i]['dateName']) :
                pass
            else:
                in_list.append(ress[i])

    # in_list.extend(new_list)

holidays = pd.DataFrame(in_list)


In [28]:
holidays.head()

,dateKind,dateName,isHoliday,locdate,seq,kst
0,01,1월1일,Y,20230101,1,NaN
1,01,설날,Y,20230121,1,NaN
2,01,설날,Y,20230122,1,NaN
3,01,설날,Y,20230123,1,NaN
4,01,대체공휴일,Y,20230124,1,NaN


In [29]:
holidays.isna().sum().sum()

82

In [30]:
holidays['locdate'] = holidays['locdate'].astype(str)

In [31]:
# type(list(holidays['locdate'])[0])

In [32]:
# '20230101' in list(holidays['locdate'])

In [33]:
# holidays['locdate'].to_datetime()
holidays['locdate'] = pd.to_datetime(holidays['locdate'], format='%Y%m%d')

In [34]:
for dd in holidays['locdate'][:5]:
    print(dd)

2023-01-01 00:00:00
2023-01-21 00:00:00
2023-01-22 00:00:00
2023-01-23 00:00:00
2023-01-24 00:00:00


In [35]:
holidays

,dateKind,dateName,isHoliday,locdate,seq,kst
0,01,1월1일,Y,2023-01-01,1,NaN
1,01,설날,Y,2023-01-21,1,NaN
2,01,설날,Y,2023-01-22,1,NaN
3,01,설날,Y,2023-01-23,1,NaN
4,01,대체공휴일,Y,2023-01-24,1,NaN
...,...,...,...,...,...,...
94,04,한식,N,2023-04-06,1,NaN
95,04,단오,N,2023-06-22,1,NaN
96,04,중복,N,2023-07-21,1,NaN
97,04,말복,N,2023-08-10,1,NaN


In [36]:
try:
    nindex = holidays.index[holidays.dateName=='기독탄신일'][0]
    holidays.iloc[nindex, 1] = '크리스마스'
except:
    pass



In [37]:
holidays[holidays.dateName=='크리스마스']

,dateKind,dateName,isHoliday,locdate,seq,kst
16,01,크리스마스,Y,2023-12-25,1,NaN


In [38]:
holidays.locdate[:5]

0   2023-01-01
1   2023-01-21
2   2023-01-22
3   2023-01-23
4   2023-01-24
Name: locdate, dtype: datetime64[ns]

In [46]:
result_dt = pd.merge(l_dt, holidays, how='left', on='locdate')
result_dt.dateKind.fillna('05', inplace=True)
result_dt.dateName.fillna('', inplace=True)
result_dt.isHoliday.fillna('N', inplace=True)
result_dt.drop(labels=['locdate', 'seq', 'kst'], axis=1, inplace=True)
result_dt

,opacity,day,dateKind,dateName,isHoliday,seq,kst
0,0.5,30,05,,N,NaN,NaN
1,,1,02,근로자의 날,N,1.0,NaN
2,,2,05,,N,NaN,NaN
3,,3,05,,N,NaN,NaN
4,,4,05,,N,NaN,NaN
5,,5,01,어린이날,Y,1.0,NaN
6,,6,03,입하,N,1.0,0319
7,,7,05,,N,NaN,NaN
8,,8,02,어버이 날,N,1.0,NaN
9,,9,05,,N,NaN,NaN


In [40]:
res_list = result_dt.to_dict('records')


In [41]:
res_list


[{'locdate': Timestamp('2023-04-30 00:00:00'),
  'opacity': '0.5',
  'day': 30,
  'dateKind': '05',
  'dateName': '',
  'isHoliday': 'N',
  'seq': nan,
  'kst': nan},
 {'locdate': Timestamp('2023-05-01 00:00:00'),
  'opacity': '',
  'day': 1,
  'dateKind': '02',
  'dateName': '근로자의 날',
  'isHoliday': 'N',
  'seq': 1.0,
  'kst': nan},
 {'locdate': Timestamp('2023-05-02 00:00:00'),
  'opacity': '',
  'day': 2,
  'dateKind': '05',
  'dateName': '',
  'isHoliday': 'N',
  'seq': nan,
  'kst': nan},
 {'locdate': Timestamp('2023-05-03 00:00:00'),
  'opacity': '',
  'day': 3,
  'dateKind': '05',
  'dateName': '',
  'isHoliday': 'N',
  'seq': nan,
  'kst': nan},
 {'locdate': Timestamp('2023-05-04 00:00:00'),
  'opacity': '',
  'day': 4,
  'dateKind': '05',
  'dateName': '',
  'isHoliday': 'N',
  'seq': nan,
  'kst': nan},
 {'locdate': Timestamp('2023-05-05 00:00:00'),
  'opacity': '',
  'day': 5,
  'dateKind': '01',
  'dateName': '어린이날',
  'isHoliday': 'Y',
  'seq': 1.0,
  'kst': nan},
 {'locda